In [ ]:
import  tqdm, json
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# 뒤에서 본 신체의 좌표 시각화
from utils.visualize import draw_joint
body_coordinates = [[0.5, 0.25], [0.4375, 0.25], [0.5625, 0.25], [0.375, 0.25], [0.625, 0.25], [0.375, 0.3], [0.625, 0.3], [0.3125, 0.35], [0.6875, 0.35], [0.25, 0.4], [0.75, 0.4], [0.375, 0.45], [0.625, 0.45], [0.375, 0.55], [0.625, 0.55], [0.375, 0.65], [0.625, 0.65], [0.5, 0.275], [0.25, 0.45], [0.75, 0.45], [0.5, 0.3], [0.5, 0.35], [0.375, 0.75], [0.625, 0.75]]
body_coord_visualization = draw_joint(np.zeros((256,192,3)).astype(np.uint8()),torch.tensor(body_coordinates))
img = np.zeros((256,192,3), dtype = np.uint8())
plt.title("Left,Center,Right")
plt.imshow(body_coord_visualization)
plt.show()


# 신경망 기반 이상치 탐지

In [ ]:
from model.pose_est import ViTpose
model_path = './experiments/PE/ViTPose_base_simple(heatmap)[model1]/'

model = ViTpose(backbone='b',mode='heatmap')
checkpoint = torch.load(f'{model_path}Last5.pth','cuda')
model.to(device)

model.load_state_dict(checkpoint['model'],strict=False)

In [ ]:
from dataset.dataset import Pose
from dataset.get_data import get_pose_dataset, split_data
dataset = get_pose_dataset("./data", anomaly="./data/anomaly")
train, valid, _ = split_data(dataset=dataset, valid_ratio=0.2, valid_selection=0) # 나뉜 부분 중 0번째 set을 validation으로 침
train_set = Pose(dataset, views = [1,2,3,4,5], training =  False,frame_mode=2)
print(len(train_set))

2998145


In [ ]:
from utils.visualize import get_xy
outputs = []
model.eval()
train_loader = DataLoader(train_set,batch_size=128,shuffle=False, num_workers=8)
with torch.no_grad():
    progress = tqdm.tqdm(train_loader,desc=f'MCPJP:')
    for i,data in enumerate(progress):
        img = data['image'].to(device)

        pred = model(img,data['view'].to(device))['heatmap'] #b,c,h,w
        pts = get_xy(pred).detach().cpu().numpy() # b,24,2
        for j in range(len(pts)):
            outputs.append(pts[j])
outputs= np.array(outputs)
np.save(f'{model_path}train',outputs)

MCPJP:: 100%|██████████| 18740/18740 [12:45:38<00:00,  2.45s/it]


In [ ]:
outputs= np.load(f'{model_path}train.npy')

In [ ]:
outputs_flip = outputs.copy()
flip_idx = [[1,2],[3,4],[5,6],[7,8],[9,10],[11,12],[13,14],[15,16],[18,19],[22,23]]

for i in tqdm.tqdm(range(len(outputs))):
    for l,r  in flip_idx :
        outputs_flip[i][r] = outputs[i][l]
        outputs_flip[i][l] = outputs[i][r]

100%|██████████| 2398615/2398615 [00:41<00:00, 57289.19it/s]


In [ ]:
from cfg import IMAGE_HEIGHT, IMAGE_WIDTH, point_name
from utils.visualize import denormalization

In [ ]:
idx_list = []
wh = [IMAGE_WIDTH,IMAGE_HEIGHT]
mcpjp = [] # original,flip
dists = [] # original,flip
count = 0
for i in range(len(train)):
    for j in range(len(train[i])):
        for k in range(len(train[i][j])):

            idx_list.append([i,j,k,count])
            count += 1

idx_list = np.array(idx_list)

for i in tqdm.tqdm(range(len(outputs))):
    view,data,frame,count = idx_list[i]

    x1,y1,x2,y2 = train[view][data][frame][2]

    #사전에 미리 Crop한 이미지 x1,y1,x2,y2
    crop_min_x = max(0,x1-200)
    crop_min_y = max(0,y1-200)
    crop_max_x = min(1920,x2+200)
    crop_max_y = min(1080,y2+200)

    true = (np.array(train[view][data][frame][3]) - [crop_min_x,crop_min_y])/[crop_max_x-crop_min_x,crop_max_y-crop_min_y]

    pred = outputs[i]
    pred_flip = outputs_flip[i]



    dist = np.sqrt(np.sum(((true-pred)*wh)**2,-1)+1e-5) # 24
    dist_flip = np.sqrt(np.sum(((true-pred_flip)*wh)**2,-1)+1e-5) # 24

    dists += [dist,dist_flip]
    mcpjp += [np.mean(dist),np.mean(dist_flip)]

dists = np.array(dists).reshape(-1,2,24).transpose(0,2,1) # n, 24, 2 (데이터 수, 관절 수, 좌우 Flip)
mcpjp = np.array(mcpjp).reshape(-1,2)  #n, 2 (데이터 수, 좌우 Flip)

In [ ]:
plt.subplot(1,2,1)
plt.title("Original")
plt.plot(mcpjp[:,0])
plt.subplot(1,2,2)
plt.title("Fliped")
plt.plot(mcpjp[:,1])
plt.show()

In [ ]:
t_dist = 20
anomaly = dists > t_dist


# 거리가 t_dist를 넘은 좌표의 수
anomaly_summation = np.sum(anomaly,1) # n, 24, 2 -> n, 2

max_dists = np.max(dists,1)
max_idx = np.argmax(dists,1)

print(anomaly.shape,anomaly_summation.shape)

for t_count in range(25):
    print(f"t_count = { t_count } \t ( {np.sum(anomaly_summation[:,0]==t_count)} / {len(anomaly_summation)} ) \t {np.sum(anomaly_summation[:,0]==t_count)/len(anomaly_summation)*100}%, \t t_count >= { t_count } \t ( {np.sum(anomaly_summation[:,0]>=t_count)} / {len(anomaly_summation)} ) \t {np.sum(anomaly_summation[:,0]>=t_count)/len(anomaly_summation)*100}%")

In [ ]:

start_idx = 0
indexing = (6 <= anomaly_summation[:,0]) & (anomaly_summation[:,0]<7)
length = len(idx_list[:len(indexing)][indexing])
for idx in tqdm.tqdm(idx_list[:len(indexing)][indexing][start_idx:start_idx+200]):
    i = idx[-1]

    print('\n',anomaly_summation[i,0],point_name[max_idx[i,0]],max_dists[i,0],i,length)


    data = train_set[i]
    img = (denormalization(data['image'].permute(1,2,0).numpy())*255).astype(np.uint8())

    plt.figure(figsize=(15,15))

    plt.subplot(1,3,1)
    # 뒤에서 본 신체의 좌표
    plt.title("Left,Center,Right")
    plt.imshow(body_coord_visualization)

    plt.subplot(1,3,2)
    plt.title("True")
    plt.imshow(draw_joint(img,data['coord'],rad=2))

    plt.subplot(1,3,3)
    plt.title("pred")
    plt.imshow(draw_joint(img,torch.from_numpy(outputs[i]),rad=2))
    plt.show()

# 모델별 신경망기반 이상치 탐지

In [ ]:
for i in range(1):
    #model_path = f'./experiments/PE/ViTPose_base_simple(heatmap)[model{i+1}]/'
    model_path = f'./experiments/PE/ViTPose_base_simple(heatmap)[model1]/'
    print(model_path)
    model = ViTpose(backbone='b',mode='heatmap')
    checkpoint = torch.load(f'{model_path}Last5.pth','cuda')
    model.to(device)

    model.load_state_dict(checkpoint['model'],strict=False)
    split_data = []
    for inner_list in dataset:
        split_data.append([data for k,data in enumerate(inner_list) if k % 5 == i])

    outputs = []
    model.eval()
    split_set = Pose(split_data, views = [1,2,3,4,5], training =  False,frame_mode=2)

    train_loader = DataLoader(split_set,batch_size=128,shuffle=False, num_workers=4)
    with torch.no_grad():
        progress = tqdm.tqdm(train_loader,desc=f'MCPJP:')
        for data in progress:
            img = data['image'].to(device)

            pred = model(img,data['view'].to(device))['heatmap'] #b,c,h,w
            pts = get_xy(pred).detach().cpu().numpy() # b,24,2
            for j in range(len(pts)):
                outputs.append(pts[j])
    outputs= np.array(outputs)
    np.save(f'{model_path}valid',outputs)

In [ ]:
sorted_data = [[],[],[],[],[]]
model_num = 0
model_path = f'./experiments/PE/ViTPose_base_simple(heatmap)[model{model_num+1},no_anomaly]/'
outputs = np.load(f'{model_path}valid.npy')
for j,inner_list in enumerate(dataset):
    sorted_data[j] += [data for k,data in enumerate(inner_list) if k % 5 == model_num]
sorted_set = Pose(sorted_data, views = [1,2,3,4,5], training =  False,frame_mode=2)
print(len(sorted_data[0]))
print(len(sorted_set))


In [ ]:
idx_list = []
wh = [IMAGE_WIDTH,IMAGE_HEIGHT]
mcpjp = [] # original
dists = [] # original
count = 0
for i in range(len(sorted_data)):
    for j in range(len(sorted_data[i])):
        for k in range(len(sorted_data[i][j][2])):

            idx_list.append([i,j,k,count])
            count += 1

idx_list = np.array(idx_list)

progress = tqdm.tqdm(range(len(outputs)))
sum_mcpjp = 0
for i in progress:
    view,data,frame,count = idx_list[i]

    x1,y1,x2,y2 = sorted_data[view][data][1]

    #사전에 미리 Crop한 이미지 x1,y1,x2,y2
    crop_min_x = max(0,x1-200)
    crop_min_y = max(0,y1-200)
    crop_max_x = min(1920,x2+200)
    crop_max_y = min(1080,y2+200)

    true = (np.array(sorted_data[view][data][2][frame]) - [crop_min_x,crop_min_y])/[crop_max_x-crop_min_x,crop_max_y-crop_min_y]

    pred = outputs[i]

    dist = np.sqrt(np.sum(((true-pred)*wh)**2,-1)+1e-5) # 24

    dists.append(dist)
    mcpjp.append(np.mean(dist))
    #sum_mcpjp += mcpjp[-2]
    #progress.set_description(f'{sum_mcpjp/(i+1)}')

dists = np.array(dists) # n,24
mcpjp = np.array(mcpjp) # n

In [ ]:

plt.title("Original")
plt.plot(mcpjp)
plt.show()

In [ ]:
#no_anomaly
t_dist = 16
anomaly = dists > t_dist

# 거리가 t_dist를 넘은 좌표의 수 구하기
anomaly_summation = np.sum(anomaly,1) # n, 24 -> n

max_dists = np.max(dists,1)
max_idx = np.argmax(dists,1)

print(anomaly.shape,anomaly_summation.shape)

for t_count in range(25):
    print(f"t_count = { t_count } \t ( {np.sum(anomaly_summation==t_count)} / {len(anomaly_summation)} ) \t {np.sum(anomaly_summation==t_count)/len(anomaly_summation)*100}%, \t t_count >= { t_count } \t ( {np.sum(anomaly_summation>=t_count)} / {len(anomaly_summation)} ) \t {np.sum(anomaly_summation>=t_count)/len(anomaly_summation)*100}%")

In [ ]:
t_dist = 16
anomaly = dists > t_dist

# 거리가 t_dist를 넘은 좌표의 수 구하기
anomaly_summation = np.sum(anomaly,1) # n, 24 -> n

max_dists = np.max(dists,1)
max_idx = np.argmax(dists,1)

print(anomaly.shape,anomaly_summation.shape)

for t_count in range(25):
    print(f"t_count = { t_count } \t ( {np.sum(anomaly_summation==t_count)} / {len(anomaly_summation)} ) \t {np.sum(anomaly_summation==t_count)/len(anomaly_summation)*100}%, \t t_count >= { t_count } \t ( {np.sum(anomaly_summation>=t_count)} / {len(anomaly_summation)} ) \t {np.sum(anomaly_summation>=t_count)/len(anomaly_summation)*100}%")

In [ ]:

labels = {model_num:{}}
indexing = (10 <= anomaly_summation) & (anomaly_summation<25)
for iter,idx in enumerate(tqdm.tqdm(idx_list[:len(indexing)][indexing])):
    i = idx[-1]
    data = sorted_set[i]
    view_idx,data_idx,frame_idx = data['idx']
    filename = sorted_data[view_idx][data_idx][0]
    label_name = sorted_data[view_idx][data_idx][-1]


    if (labels[model_num].get(filename) is None):
        labels[model_num][filename] = {"class": data["class"],"label":label_name,"frame":[frame_idx]}
    else:
        labels[model_num][filename]["frame"].append(frame_idx)


with open(f".//data/anomaly/{model_num}.json",'w') as f:
    json.dump(labels,f)


In [ ]:
print(len(labels.keys()))
labels

In [ ]:
# no_anomaly
start_idx = 0
indexing = (10 <= anomaly_summation) & (anomaly_summation<25)
length = len(idx_list[:len(indexing)][indexing])
for idx in tqdm.tqdm(idx_list[:len(indexing)][indexing][start_idx:start_idx+200]):
    i = idx[-1]

    print('\n',anomaly_summation[i],point_name[max_idx[i]],max_dists[i],i,length)


    data = sorted_set[i]
    img = (denormalization(data['image'].permute(1,2,0).numpy())*255).astype(np.uint8())

    plt.figure(figsize=(15,15))

    plt.subplot(1,3,1)
    # 뒤에서 본 신체의 좌표
    plt.title("Left,Center,Right")
    plt.imshow(body_coord_visualization)

    plt.subplot(1,3,2)
    plt.title("True")
    plt.imshow(draw_joint(img,data['coord'],rad=2))

    plt.subplot(1,3,3)
    plt.title("pred")
    plt.imshow(draw_joint(img,torch.from_numpy(outputs[i]),rad=2))
    plt.show()

In [ ]:
# anomaly
start_idx = 0
indexing = (10 <= anomaly_summation) & (anomaly_summation<25)
length = len(idx_list[:len(indexing)][indexing])
for idx in tqdm.tqdm(idx_list[:len(indexing)][indexing][start_idx:start_idx+200]):
    i = idx[-1]

    print('\n',anomaly_summation[i],point_name[max_idx[i]],max_dists[i],i,length)


    data = sorted_set[i]
    img = (denormalization(data['image'].permute(1,2,0).numpy())*255).astype(np.uint8())

    plt.figure(figsize=(15,15))

    plt.subplot(1,3,1)
    # 뒤에서 본 신체의 좌표
    plt.title("Left,Center,Right")
    plt.imshow(body_coord_visualization)

    plt.subplot(1,3,2)
    plt.title("True")
    plt.imshow(draw_joint(img,data['coord'],rad=2))

    plt.subplot(1,3,3)
    plt.title("pred")
    plt.imshow(draw_joint(img,torch.from_numpy(outputs[i]),rad=2))
    plt.show()

In [ ]:
for model_num in range(5):
    sorted_data = [[],[],[],[],[]]
    model_path = f'./experiments/PE/ViTPose_base_simple(heatmap)[model{model_num+1}]/'
    outputs = np.load(f'{model_path}valid.npy')
    for j,inner_list in enumerate(dataset):
        sorted_data[j] += [data for k,data in enumerate(inner_list) if k % 5 == model_num]
    sorted_set = Pose(sorted_data, views = [1,2,3,4,5], training =  False,frame_mode=2)
    print(model_num)
    print(len(sorted_data[0]))
    print(len(sorted_set))
    idx_list = []
    wh = [IMAGE_WIDTH,IMAGE_HEIGHT]
    mcpjp = [] # original
    dists = [] # original
    count = 0
    for i in range(len(sorted_data)):
        for j in range(len(sorted_data[i])):
            for k in range(len(sorted_data[i][j][2])):

                idx_list.append([i,j,k,count])
                count += 1

    idx_list = np.array(idx_list)

    progress = tqdm.tqdm(range(len(outputs)))
    sum_mcpjp = 0
    for i in progress:
        view,data,frame,count = idx_list[i]

        x1,y1,x2,y2 = sorted_data[view][data][1]

        #사전에 미리 Crop한 이미지 x1,y1,x2,y2
        crop_min_x = max(0,x1-200)
        crop_min_y = max(0,y1-200)
        crop_max_x = min(1920,x2+200)
        crop_max_y = min(1080,y2+200)

        true = (np.array(sorted_data[view][data][2][frame]) - [crop_min_x,crop_min_y])/[crop_max_x-crop_min_x,crop_max_y-crop_min_y]

        pred = outputs[i]

        dist = np.sqrt(np.sum(((true-pred)*wh)**2,-1)+1e-5) # 24

        dists.append(dist)
        mcpjp.append(np.mean(dist))
        #sum_mcpjp += mcpjp[-2]
        #progress.set_description(f'{sum_mcpjp/(i+1)}')

    dists = np.array(dists) # n,24
    mcpjp = np.array(mcpjp) # n
    t_dist = 16
    anomaly = dists > t_dist

    # 거리가 t_dist를 넘은 좌표의 수 구하기
    anomaly_summation = np.sum(anomaly,1) # n, 24 -> n

    max_dists = np.max(dists,1)
    max_idx = np.argmax(dists,1)

    print(anomaly.shape,anomaly_summation.shape)

    labels = {model_num:{}}
    indexing = (10 <= anomaly_summation) & (anomaly_summation<25)
    for iter,idx in enumerate(tqdm.tqdm(idx_list[:len(indexing)][indexing])):
        i = idx[-1]
        data = sorted_set[i]
        view_idx,data_idx,frame_idx = data['idx']
        filename = sorted_data[view_idx][data_idx][0]
        label_name = sorted_data[view_idx][data_idx][-1]


        if (labels[model_num].get(filename) is None):
            labels[model_num][filename] = {"class": data["class"],"label":label_name,"frame":[frame_idx]}
        else:
            labels[model_num][filename]["frame"].append(frame_idx)


    with open(f"./data/anomaly/{model_num}.json",'w') as f:
        json.dump(labels,f)
